# Библиотеки

In [48]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
import re
import math
from statsmodels.stats.descriptivestats import sign_test
from arch.bootstrap import IIDBootstrap
import itertools

# Код генерации выборки

In [2]:
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code%(2**32-1))
    ars = np.random.RandomState(code%(2**16-1))
    if rs.randint(low=0, high=2) == 0:
        while True:
            data = rs.randn(20)
            if st.shapiro(data)[1] > 0.2 or ars.binomial(100, 0.05) > 10:
                break
    else:
        while True:
            data = np.concatenate([rs.randn(6) - 2, rs.randn(6) + 2])
            if st.shapiro(data)[1] < 0.01 or ars.binomial(100, 0.05) > 10:
                break
    return data

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [3]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: bogdanov.ai@phystech.edu


{'mail': 'bogdanov.ai@phystech.edu',
 'id': 1162440694,
 'data': array([-2.30553324, -2.56678427, -1.30157427, -0.90646724, -1.77345706,
        -0.73034251,  2.59815099,  2.28058966,  1.13836052,  0.66810466,
         3.20580377,  1.23182388])}

# Работа с выборкой

In [26]:
data = task['data']

## Применим ли t-критерий для проверки равенства среднего нулю для данной выборки?

In [27]:
if st.shapiro(data).pvalue < 0.05:
    print(False)
else:
    print(True)

True


## p-value для критерия Уилкоксона (m0=0)

In [28]:
st.wilcoxon(data).pvalue

0.90966796875

## p-value для критерия знаков (m0=0)

In [63]:
_, pvalue = sign_test(data)
print(pvalue)

1.0


## Рассмотрим перестановочный критерий для среднего. Сколько перестановок возможно для данной выборки?

In [66]:
len(list(itertools.product([-1, 1], repeat=len(data))))

4096

## Рассмотрим перестановочный критерий для проверки среднего (m0=0.5). Для какого количества перестановок статистика T будет иметь значение меньшее или равное статистике для оригинальной выборки?

In [61]:
t = (data - 0.5)
T = np.sum(t)

count = 0
for v in itertools.product([-1, 1], repeat=len(data)):
    if np.dot(t, v)<=T:
        count+=1
    
print(good)

1065


## Левая граница доверительного интервала для среднего выборки

In [43]:
bs = IIDBootstrap(data)
print(bs.conf_int(np.mean, 1000, method='bca')[0])

[-0.89887803]


## Правая граница доверительного интервала для среднего выборки

In [42]:
bs = IIDBootstrap(data)
print(bs.conf_int(np.mean, 1000, method='bca')[1])

[1.29027939]
